In [1]:
import requests
import numpy as np
import pandas as pd
from lxml import etree, html
from html.parser import HTMLParser
import re
import matplotlib.pylab as plt


In [2]:
url = 'https://www.fantapiu3.com/fantacalcio-storico-voti-corriere-sport-2016-2017.php'
# url = 'https://www.fantapiu3.com/fantacalcio-storico-voti-fantagazzetta-2018-2019.php'
headers = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 11_0 like Mac OS X) AppleWebKit/604.1.34 (KHTML, like Gecko) Version/11.0 Mobile/15A5341f Safari/604.1'}
r = requests.get(url, headers=headers)
html = r.text
f = open('output.html', 'w+')
f.write(html)
f.close()

In [3]:
header = ['dots','giocatore', 'squadra', '?', 'ruolo', 'fantavoto', 'voto', 'gol_fatti_subiti', 
          'ammonizioni', 'espulsioni', 'rigori_parati_segnati', 'autogol', 'portiere_imbattuto']
data_types={'dots':int,'giocatore':str, 'squadra':str, '?':str, 'ruolo':str, 'fantavoto':float, 
           'voto':float, 'gol_fatti_subiti':float, 
          'ammonizioni':float, 'espulsioni':float, 'rigori_parati_segnati':float, 
           'autogol':float, 'portiere_imbattuto':float}

In [4]:
html = html.replace('\n', '').replace('\t', '')

In [5]:

html_list = html.split('</tr>')
records = []
for i in range(len(html_list)):
    try:
        entry = html_list[i].replace('</td>', '').replace('<!--', '').replace('-->', '').replace(',', '.').split('<td>')
        entry[0] = re.search('GIORNATA (\d+)',entry[0]).group(1)
        record = {}
        for i, e in enumerate(entry): record[header[i]] = 0 if len(e)==0 or e=='-' else e
        records.append(record)
    except AttributeError:
        print(i,entry[0])
        print()


1                       <tr name="theader">                        <th class="game-player-result__date">Gio</th>                        <th class="game-player-result__vs">Sq</th>                        <th class="game-player-result__score">Ru</th>                        <th class="game-player-result__stat">FV</th>                        <th class="game-player-result__stat">V</th>                        <th class="game-player-result__stat">GF/GS</th>                        <th class="game-player-result__stat">Amm</th>                        <th class="game-player-result__stat">Esp</th>                        <th class="game-player-result__stat">RP/RS</th><th class="game-player-result__stat">Aut</th>                        <th class="game-player-result__stat">PI</th>                      

9742                     </tbody>                  </table>                </div>              </div>              </div>             Post Sharing Buttons             <div class="post-sharing">        

In [6]:
df = pd.DataFrame(data=records, columns=header, ).sort_values(by='fantavoto')
df = df.astype(data_types)
df = df.drop('?', axis=1)
df = df[df.giocatore!='nan']

df

,dots,giocatore,squadra,ruolo,fantavoto,voto,gol_fatti_subiti,ammonizioni,espulsioni,rigori_parati_segnati,autogol,portiere_imbattuto
29,23,BIZZARRI A,Pescara,P,-1.0,5.0,-6.0,0.0,0.0,0.0,0.0,0.0
243,27,GABRIEL,Cagliari,P,-1.0,4.0,-5.0,0.0,0.0,0.0,0.0,0.0
657,12,STORARI M,Cagliari,P,-1.0,4.0,-5.0,0.0,0.0,0.0,0.0,0.0
28,24,BIZZARRI A,Pescara,P,-1.0,4.0,-5.0,0.0,0.0,0.0,0.0,0.0
457,33,POSAVEC J,Palermo,P,-1.0,5.0,-6.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9464,20,SIMEONE G,Genoa,A,9.5,6.5,3.0,0.0,0.0,0.0,0.0,0.0
9463,21,SIMEONE G,Genoa,A,9.5,6.5,3.0,0.0,0.0,0.0,0.0,0.0
9462,6,SIMEONE G,Genoa,A,9.5,6.5,3.0,0.0,0.0,0.0,0.0,0.0
4244,15,BARRETO E,Sampdoria,C,9.5,7.0,3.0,-0.5,0.0,0.0,0.0,0.0


In [7]:
df.fantavoto.max()


21.0